In [ ]:
import pandas as pd
from collections import defaultdict

# --- Unified file list: (file_path, snapshot_name, table_name) ---
all_files = [
    # Assemblies
    (r"data\PreNexPartAssemblies.csv",           "PreNex",  "assemblies"),
    (r"data\PartAssemblies_25_06_25.csv",         "PostNex", "assemblies"),
    (r"data\PartAssembliesNGD27_30_06_25.csv",    "NGD27",   "assemblies"),
    (r"data\PartAssembliesDammagedPost27.csv",    "Post27",  "assemblies"),

    # Revisions
    (r"data\PreNexPartRevisions.csv",             "PreNex",  "revisions"),
    (r"data\PartRevisions_25_06_25.csv",           "PostNex", "revisions"),
    (r"data\PartRevisionsNGD27_30_06_25.csv",      "NGD27",   "revisions"),
    (r"data\PartRevisionsDammagedPost27.csv",      "Post27",  "revisions"),

    # Materials
    (r"data\PreNexPartMaterials.csv",              "PreNex",  "materials"),
    (r"data\PartMaterials_25_06_25.csv",            "PostNex", "materials"),
    (r"data\PartMaterialsNGD27_30_06_25.csv",       "NGD27",   "materials"),
    (r"data\PartMaterialsDammagedPost27.csv",       "Post27",  "materials"),
]

# --- Load into nested dictionary: dfs_snapshots[snapshot][table] ---
dfs_snapshots = defaultdict(dict)

DATE_FORMAT = "%d/%m/%Y %I:%M:%S %p"  # dd/mm/yyyy h:mm:ss AM/PM

def parse_mixed_dates(series: pd.Series) -> pd.Series:
    """
    Parse a series of date strings that may be in either:
      - "%d/%m/%Y %I:%M:%S %p"  (12h + AM/PM)
      - "%d/%m/%Y %H:%M"        (24h, no AM/PM)
    """
    # First pass: 12-hour clock with AM/PM
    dt = pd.to_datetime(
        series,
        format="%d/%m/%Y %I:%M:%S %p",
        errors="coerce",
        dayfirst=True,
    )

    # Any failures -> try 24-hour clock
    mask = dt.isna() & series.notna()
    if mask.any():
        dt.loc[mask] = pd.to_datetime(
            series[mask],
            format="%d/%m/%Y %H:%M",
            errors="coerce",
            dayfirst=True,
        )
    return dt

def load_csv_with_dates(path: str) -> pd.DataFrame:
    # 1) Peek header to find date columns
    cols = pd.read_csv(path, nrows=0).columns
    date_cols = [c for c in cols if c.endswith("CreatedDate")]

    # 2) Read all as strings for date cols (faster CSV read)
    dtype = {c: pd.StringDtype() for c in date_cols}
    df = pd.read_csv(path, dtype=dtype)

    # 3) Convert detected date columns using mixed parser
    for c in date_cols:
        df[c] = parse_mixed_dates(df[c])

    return df

for path, snapshot, table in all_files:
    dfs_snapshots[snapshot][table] = load_csv_with_dates(path)

#ast to regular dict if you're done mutating
dfs_snapshots = dict(dfs_snapshots)


In [ ]:
for file_path, snapshot_name, table_name in all_files:
    print(dfs_snapshots[snapshot_name][table_name].columns.tolist())

In [ ]:
# Define index columns per table
index_map = {
    "assemblies": ["imaPartID", "imaPartRevisionID", 'imaMethodID', 'imaMethodRevisionID', 'imaMethodAssemblyID'],
    "revisions": ["imrPartID", "imrPartRevisionID", "imrCreatedBy"],
    "materials": ["immPartID", "immPartRevisionID", 'immMethodID', 'immMethodRevisionID', 'immMethodAssemblyID','immMethodMaterialID']
}

# Loop snapshots and tables
for snapshot, tables in dfs_snapshots.items():
    for table, df in tables.items():
        if table in index_map:
            dfs_snapshots[snapshot][table] = df.set_index(index_map[table])


In [ ]:
#test check for column names and index names
for snapshot, tables in dfs_snapshots.items():
    for table, df in tables.items():
        if table == 'revisions':
            print(dfs_snapshots[snapshot][table].columns.tolist())
            print(dfs_snapshots[snapshot][table].index.names)
            print(dfs_snapshots[snapshot][table].dtypes)

In [ ]:
import pandas as pd
import numpy as np

#"report" (default): don’t change data; just return dupe reports (no diff if dupes exist).
#"sequence": add a per-key __seq__ to disambiguate multiplicity and proceed.
#"drop_exact": drop exact duplicate rows (content-identical) before diffing.

def _dupe_counts(df, key_cols):
    g = df.reset_index()[key_cols].value_counts(sort=False).rename("count")
    return g[g > 1].sort_index()

def _add_seq_index(df, key_cols, seq_name="__seq__"):
    # Keep original order per key; assign 0..n-1
    if list(df.index.names) == key_cols:
        df = df.reset_index()
    df = df.copy()
    df[seq_name] = df.groupby(key_cols).cumcount()
    return df.set_index(key_cols + [seq_name])

def _drop_exact_dupes(df, key_cols):
    return df.reset_index().drop_duplicates().set_index(key_cols)

def _row_hashes(df):
    # Stable, vectorized row hash (handles NaNs)
    h = np.zeros(len(df), dtype=np.uint64)
    for c in df.columns:
        h ^= pd.util.hash_pandas_object(df[c], index=False).to_numpy(dtype=np.uint64, copy=False)
    return h

def normalize_for_diff(df):
    out = df.copy()

    # IDs
    if "imrPartID" in out.columns:
        out["imrPartID"] = out["imrPartID"].astype("string").str.strip()

    if "imrPartRevisionID" in out.columns:
        out["imrPartRevisionID"] = (
            out["imrPartRevisionID"]
            .astype("string")
            .str.strip()
            .fillna("")  # sentinel for missing revision
        )

    # Normalize strings
    str_cols = out.select_dtypes(include=["object", "string"]).columns
    out[str_cols] = out[str_cols].apply(
        lambda s: s.astype("string").str.strip().replace({"": pd.NA})
    )

    # Normalize dates
    for c in out.columns:
        if c.endswith("CreatedDate"):
            out[c] = pd.to_datetime(out[c], errors="coerce").dt.floor("min")

    return out
    
def fast_anycol_diff(
    pre,
    post,
    key_cols,
    label_pre="PreNex",
    label_post="PostNex",
    duplicate_strategy="report",  # 'report' | 'sequence' | 'drop_exact'
):
    """
    Fast diff across ANY columns, with duplicate detection/handling.

    Returns dict:
      - changed_diff, changed_side_by_side, new_rows, removed_rows
      - duplicates_pre, duplicates_post, duplicate_count_diff
      - meta: info about strategy applied
    """
     # --- normalize snapshots
    df_pre = normalize_for_diff(pre)
    df_post = normalize_for_diff(post)

    # Ensure indexed by key (no copy if already correct)
    if list(df_pre.index.names) != key_cols:
        df_pre = df_pre.set_index(key_cols, drop=True)
    if list(df_post.index.names) != key_cols:
        df_post = df_post.set_index(key_cols, drop=True)

    # --- Duplicate detection
    dup_pre  = _dupe_counts(df_pre,  key_cols)
    dup_post = _dupe_counts(df_post, key_cols)

    # Differences in multiplicity (including 0→n, n→0)
    cnt_pre  = df_pre.reset_index()[key_cols].value_counts(sort=False).rename("count_pre")
    cnt_post = df_post.reset_index()[key_cols].value_counts(sort=False).rename("count_post")
    duplicate_count_diff = (
        cnt_pre.to_frame().join(cnt_post.to_frame(), how="outer").fillna(0).astype(int)
    )
    duplicate_count_diff = duplicate_count_diff[
        (duplicate_count_diff["count_pre"] != 1) | (duplicate_count_diff["count_post"] != 1)
    ].sort_index()

    # Optionally resolve before diff
    resolved = False
    if duplicate_strategy == "sequence":
        df_pre = _add_seq_index(df_pre, key_cols)
        df_post = _add_seq_index(df_post, key_cols)
        key_cols = df_pre.index.names  # now includes '__seq__' where applied
        resolved = True
    elif duplicate_strategy == "drop_exact":
        if not dup_pre.empty:
            df_pre = _drop_exact_dupes(df_pre, key_cols)
        if not dup_post.empty:
            df_post = _drop_exact_dupes(df_post, key_cols)
        resolved = True
    elif duplicate_strategy != "report":
        raise ValueError("duplicate_strategy must be 'report', 'sequence', or 'drop_exact'")

    # If reporting only and duplicates exist, return reports without diffing
    if duplicate_strategy == "report" and (not dup_pre.empty or not dup_post.empty):
        return {
            "changed_diff": pd.DataFrame(),
            "changed_side_by_side": pd.DataFrame(),
            "new_rows": pd.DataFrame(),
            "removed_rows": pd.DataFrame(),
            "duplicates_pre": dup_pre,
            "duplicates_post": dup_post,
            "duplicate_count_diff": duplicate_count_diff,
            "meta": {"duplicate_strategy": "report", "diff_performed": False},
        }

    # --- Proceed with diff (unique rows per key at this point)
    common  = df_pre.index.intersection(df_post.index)
    added   = df_post.index.difference(df_pre.index)
    removed = df_pre.index.difference(df_post.index)

    cols = df_pre.columns.union(df_post.columns)
    pre_common  = df_pre.loc[common].reindex(columns=cols)
    post_common = df_post.loc[common].reindex(columns=cols)

    # Hash rows to find changed keys fast
    h_pre  = _row_hashes(pre_common)
    h_post = _row_hashes(post_common)
    changed_mask = h_pre != h_post
    if changed_mask.any():
        changed_idx = pre_common.index[changed_mask]
        changed_side_by_side = pd.concat(
            [
                pre_common.loc[changed_idx].add_suffix(f"_{label_pre}"),
                post_common.loc[changed_idx].add_suffix(f"_{label_post}"),
            ],
            axis=1,
        ).reset_index()
        changed_diff = post_common.loc[changed_idx].compare(
            pre_common.loc[changed_idx],
            align_axis=1,
            keep_equal=False,
            result_names=(label_post, label_pre),
        )
    else:
        changed_side_by_side = pd.DataFrame(columns=list(key_cols))
        changed_diff = pd.DataFrame()

    new_rows     = df_post.loc[added].reset_index()
    removed_rows = df_pre.loc[removed].reset_index()

    return {
        "changed_diff": changed_diff,
        "changed_side_by_side": changed_side_by_side,
        "new_rows": new_rows,
        "removed_rows": removed_rows,
        "duplicates_pre": dup_pre,
        "duplicates_post": dup_post,
        "duplicate_count_diff": duplicate_count_diff,
        "meta": {"duplicate_strategy": "sequence" if resolved else duplicate_strategy, "diff_performed": True},
    }


In [ ]:
key = ['imrPartID','imrPartRevisionID']
pre = dfs_snapshots['PreNex']['revisions']
post = dfs_snapshots['NGD27']['revisions']
def dup_report(df, name):
    dups = df.index.duplicated(keep=False)
    print(f"{name}: total={len(df)}, dup_rows={dups.sum()}, dup_keys={df.index[dups].nunique()}")

dup_report(pre,  "PreNex")
dup_report(post, "NGD27")

# See some examples
#print(pre[pre.index.duplicated(keep=False)].head(10))
#print(post[post.index.duplicated(keep=False)].head(10))

#print(pre.loc["VSTS2MACK01"])
#print(post.loc["VSTS2MACK01"])


In [ ]:
#PreNex, PostNex, NGD27, Post27    
check = 'revisions'
pre = dfs_snapshots['PreNex'][check]
post = dfs_snapshots['PostNex'][check]
#print(pre.head())
#print(post.head())
out = fast_anycol_diff(pre, post, index_map[check], duplicate_strategy="sequence")
# out['changed_diff'], out['changed_side_by_side'], out['new_rows'], out['removed_rows']
print(out['changed_side_by_side'].to_string())


In [ ]:
#PreNex, PostNex, NGD27, Post27    
check = 'materials'
pre = dfs_snapshots['PreNex'][check]
post = dfs_snapshots['Post27'][check]
#print(pre.head())
#print(post.head())
out = fast_anycol_diff(pre, post, index_map[check], duplicate_strategy="sequence")
# out['changed_diff'], out['changed_side_by_side'], out['new_rows'], out['removed_rows']
print(out['removed_rows'].to_string())


In [ ]:
#PreNex, PostNex, NGD27, Post27    
check = 'assemblies'
pre = dfs_snapshots['PreNex'][check]
post = dfs_snapshots['Post27'][check]
#print(pre.head())
#print(post.head())
out = fast_anycol_diff(pre, post, index_map[check], duplicate_strategy="sequence")
# out['changed_diff'], out['changed_side_by_side'], out['new_rows'], out['removed_rows']
print(out['changed_side_by_side'].to_string())
